In [1]:
data_var = '2024-02-15'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12221,2024-02-15,Eua Ncaa,22:00,James Madison,Georgia State,1.11,6.62,153.5,1.80,1.95,-14.5,2.00,1.71,zq4E7KCP,0.900901,0.151057,0.555556,0.512821,0.051958,111.886,26.917513,0.240580,2.4,1.341641,0.559017,102.20,1.396,0.413799,0.296418,65.0,177.068,72.166050,0.407561,1.2,1.643168,1.369306,102.20,2.340,0.937230,0.400526,13.0,73,73,1.40,1.40,91.872,198.340,1.008062,0.056569,0.110545,0.64,0.128,0.859375,0.893011,0.9,0.006989,-1.28,-0.256,-21.953125,0.435722,0.2,-0.235722
12222,2024-02-15,Eua Ncaa,22:00,Nebraska O.,South Dakota Coyotes,1.27,4.06,151.5,1.91,1.91,-9.5,2.00,1.71,OpSqRZod,0.787402,0.246305,0.523560,0.523560,0.033707,187.568,45.726702,0.243787,1.2,1.643168,1.369306,180.12,2.418,0.296176,0.122488,-7.0,369.628,258.278090,0.698751,0.6,1.341641,2.236068,205.16,4.674,3.527681,0.754746,-36.0,79,92,2.28,2.23,137.990,441.810,0.740273,0.000000,0.110545,2.21,0.442,0.610860,0.000000,0.0,0.000000,-5.00,-1.000,-3.060000,0.247389,0.2,-0.047389
12223,2024-02-15,Eua Ncaa,22:00,North Texas,Memphis,1.87,1.95,135.5,1.80,1.95,-2.5,2.05,1.71,OCtsgt7H,0.534759,0.512821,0.555556,0.512821,0.047580,149.260,83.356705,0.558466,0.6,1.341641,2.236068,186.32,2.270,1.337217,0.589082,-1.0,103.584,26.750915,0.258253,1.8,1.643168,0.912871,117.00,1.286,0.183793,0.142919,9.0,68,90,2.74,1.30,97.492,129.506,0.029617,0.056569,0.127881,-0.52,-0.104,-8.365385,0.679806,0.6,-0.079806,-0.92,-0.184,-5.163043,0.594325,0.7,0.105675
12224,2024-02-15,Eua Ncaa,22:00,Oral Roberts,North Dakota St.,1.51,2.64,145.5,1.91,1.91,-5.5,2.00,1.71,pfTuSFWk,0.662252,0.378788,0.523560,0.523560,0.041040,187.138,68.344135,0.365207,1.8,1.643168,0.912871,285.84,2.546,1.105862,0.434353,-15.0,189.814,80.664858,0.424968,1.8,1.643168,0.912871,135.27,2.682,1.163473,0.433808,-8.0,72,81,3.97,1.67,131.960,215.172,0.385075,0.000000,0.110545,1.50,0.300,1.700000,0.000000,0.0,0.000000,1.42,0.284,5.774648,0.311211,0.2,-0.111211
12225,2024-02-15,Eua Ncaa,22:00,Radford,Winthrop,2.05,1.80,139.5,1.80,1.95,1.5,1.91,1.91,AZ14uQnf,0.487805,0.555556,0.555556,0.512821,0.043360,127.914,44.487750,0.347794,1.2,1.643168,1.369306,154.03,1.832,0.620580,0.338745,-6.0,131.734,35.151401,0.266836,1.2,1.643168,1.369306,148.00,1.728,0.394677,0.228401,24.0,73,74,2.11,2.00,96.734,135.794,0.091832,0.056569,0.000000,-1.38,-0.276,-3.804348,0.000000,0.0,0.000000,-2.14,-0.428,-1.869159,0.476875,0.4,-0.076875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12345,2024-02-15,Eua Ncaa,22:30,Siu Edwardsville,Lindenwood,1.18,4.89,131.5,1.83,1.83,-9.5,1.83,1.83,S4e1pP7O,0.847458,0.204499,0.546448,0.546448,0.051957,157.398,127.948967,0.812901,1.2,1.643168,1.369306,380.80,2.316,1.846220,0.797159,-8.0,191.968,144.987405,0.755269,0.6,1.341641,2.236068,93.81,3.054,2.052323,0.672011,-32.0,68,53,5.60,1.77,106.550,340.120,0.864371,0.000000,0.000000,-0.81,-0.162,-1.111111,0.684590,0.8,0.115410,-2.62,-0.524,-7.423664,0.245361,0.4,0.154639
12346,2024-02-15,Eua Ncaa,22:30,Purdue,Minnesota,1.05,9.30,146.5,1.83,1.83,-17.5,1.83,1.83,ShHdRz2k,0.952381,0.107527,0.546448,0.546448,0.059908,98.696,16.850505,0.170731,3.0,0.000000,0.000000,82.16,1

## Apostas do dia

### Back Home V3 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
12250,13:30,Israel Wbl Feminina,Maccabi Urban F,Hapoel Jerusalem F,1.79,Back Home
12254,15:04,Suécia Basketligan,Sodertalje,Nassjo,2.61,Back Home
12285,21:00,Eua Ncaa,East Carolina,Wichita State,1.77,Back Home
12292,01:00,Eua Ncaa,Fresno State,UNLV,2.69,Back Home
12305,07:00,Coreia Do Sul Kbl,Anyang JungKwanJang,KoGas,1.77,Back Home
12317,22:00,Eua Ncaa,Texas Arlington,Tarleton,1.55,Back Home
12330,23:00,Eua Ncaa,Washington,Stanford,1.55,Back Home
